In [1]:
%%capture

# Install the SDK (Uncomment the code if the SDK is not installed before)
!python3 -m pip install 'kfp' --quiet
!python3 -m pip install pandas --upgrade -q

In [ ]:
!pip3 install kfp --upgrade --user

In [12]:
PROJECT_ID = ''
IMAGE_NAME = ''

In [ ]:
!docker build . -t gcr.io/$PROJECT_ID/$IMAGE_NAME:latest

In [ ]:
!docker push gcr.io/$PROJECT_ID/$IMAGE_NAME:latest

In [32]:
FULL_IMAGE_NAME = ''

In [33]:
%env FULL_IMAGE_NAME={FULL_IMAGE_NAME}

env: FULL_IMAGE_NAME=gcr.io/camus-mlops/census-keras@sha256:6a088a18f22b244bcf753c8f7327d489ed4b844e0a19694b8afc649f8c78d1b3


In [35]:
%%writefile ./pipeline/census_training_pipeline.py

"""Simple KFP with one component."""

import os

import kfp
from kfp import dsl
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

FULL_IMAGE_NAME = os.getenv('FULL_IMAGE_NAME')

# Create Training Component

def census_train_op():
    return dsl.ContainerOp(
        name='Census Training Component',
        image=FULL_IMAGE_NAME)

@kfp.dsl.pipeline(
    name='Census Keras Training',
    description='The pipeline training a census model.'
)
def census_train(project_id, region):
    """Orchestrates training and deployment of a keras model."""
    census_train_op()

Overwriting ./pipeline/census_training_pipeline.py


In [36]:
!dsl-compile --py pipeline/census_training_pipeline.py --output census_pipeline.yaml